In [1]:
import sys
import os
import numpy as np
from tqdm import tqdm
import datetime

In [21]:
# vor_path = "/home/ubuntu/git/active_vertex"
vor_path = 'C:\\Users\\Pranav\\git\\active_vertex'
sys.path.append(vor_path)

import voronoi_model.voronoi_model_periodic as avm
import voronoi_model.voronoi_model_periodic_dev as avm_dev

In [3]:
def print_SPV_params(vor2, p, J):
    print(
        """
    Number of cells : {0}
    Time-span       : {1:.2f} - {2:.2f}
    Time-step (dt)  : {3:.3f}

    === Parameters ===
    Dr      : {4:.1e}
    kappa_A : {5:.2f}
    kappa_P : {6:.2f}
    a       : {7:.2f}
    k       : {8:.1f}
    p0      : {9:.2f}
    v0      : {10:.2e}
    J (adh) : {11:.3f}
    """.format(
            vor2.n_c,
            vor2.t_span[0],
            vor2.t_span[-1],
            vor2.dt,
            vor2.Dr,
            vor2.kappa_A,
            vor2.kappa_P,
            vor2.a,
            vor2.k,
            p,
            vor2.v0,
            J
        )
    )

In [4]:
f = 200
t0 = 0
tmax = 1
dt = 0.05
n_t = int((tmax - t0) * f / dt) + 1  # calculates the n_t to get the desired dt

a = 0.35
k = 2

In [5]:
p = 3.8
J = 0
v = 1

In [6]:
# vor2 = avm.Tissue()
vor2 = avm_dev.Tissue()
vor2.generate_cells(600)
vor2.make_init(10)

vor2.set_GRN_t_span(t0, tmax, n_t, scaling_factor=f);
vor2.v0 = v
vor2.n_warmup_steps = int(200 / dt)

W = J * np.array([[1, 0], [0, 1]])
vor2.set_interaction(W=W, pE=0)
vor2.A0 = 0.86
vor2.P0 = p * np.sqrt(vor2.A0)

vor2.Dr = 0.01
vor2.kappa_A = 0.2
vor2.kappa_P = 0.1
vor2.a = a
vor2.k = k

In [7]:
print_SPV_params(vor2, p, J)


    Number of cells : 120
    Time-span       : 0.00 - 200.00
    Time-step (dt)  : 0.050

    === Parameters ===
    Dr      : 1.0e-02
    kappa_A : 0.20
    kappa_P : 0.10
    a       : 0.35
    k       : 2.0
    p0      : 3.80
    v0      : 1.00e+00
    J (adh) : 0.000
    


In [8]:
int(200 / dt)

4000

In [9]:
vor2.n_warmup_steps = 1
# vor2.warmup_SPV(F_get = vor2.get_F_periodic)

In [10]:
# vor2.x_save
# vor2.step_voronoi(x, )

In [11]:
vor2.simulate(progress_bar=True, print_updates=True);

Warming up SPV (1 steps)


 69%|██████▊   | 2741/4000 [00:31<00:17, 71.39it/s]

timeout: 19:13:23


 76%|███████▋  | 3057/4000 [00:35<00:10, 86.02it/s]


get_F_periodic encountered nan:
            nans?
            False
            False
            True
OOPSIE: 19:13:23
F_get: get_F_periodic
                Has nan?
                F:               True
                self.neighbours: False
                self.vs:         False
                
                


FloatingPointError: invalid value encountered in remainder

In [19]:
args = [
    vor2.tris,
    vor2.CV_matrix,
    vor2.n_v,
    vor2.n_c,
    vor2.L,
    vor2.A,
    vor2.P,
    vor2.Cents,
    vor2.kappa_A,
    vor2.kappa_P,
    vor2.A0,
    vor2.P0,
]

has_nan = []
for i, a in enumerate(args):
    if np.any(np.isnan(a)):
        has_nan.append(i)

In [20]:
has_nan

[]

In [ ]:
vor2.animate("f200_tmax2", n_frames=100, print_updates=False)

OK!! So when equiangulate times out, self.get_F_periodic(...) calls get_F_periodic(\*args), which returns nans - even when none of the args contain nans!!

<hr>

In [1]:
import sys
import os
import numpy as np
from tqdm import tqdm
import datetime

In [2]:
# vor_path = "/home/ubuntu/git/active_vertex"
vor_path = 'C:\\Users\\Pranav\\git\\active_vertex'
sys.path.append(vor_path)

import voronoi_model.voronoi_model_periodic as avm

In [3]:
p_space = np.linspace(3.5, 3.9, 5)
J_space = [0]
v_space = np.logspace(-2, 0, 5)[1:]

param_space = np.meshgrid(
    p_space, 
    J_space,
    v_space,
)
param_space = np.array(param_space).T.reshape(-1, 3)

In [4]:
param_space.shape

(20, 3)

In [47]:
param_space = np.array([
    [3.8, 0, 0.2],
    [3.5, 0, 0.2]
])

In [48]:
iterator = param_space
iterator = tqdm(iterator)

for p, J, v in iterator:
    prefix = f"p0{p:.2f}_J{J:.2f}_v0{v:.2e}"

    vor2 = avm.Tissue()
    vor2.generate_cells(600)
    vor2.make_init(10)

    vor2.set_GRN_t_span(t0, tmax, n_t, scaling_factor=f);
    vor2.v0 = v / f
    vor2.n_warmup_steps = int(200 / dt)

    W = J * np.array([[1, 0], [0, 1]])
    vor2.set_interaction(W=W, pE=0)
    vor2.A0 = 0.86
    vor2.P0 = p * np.sqrt(vor2.A0)

    vor2.Dr = 0.01
    vor2.kappa_A = 0.2
    vor2.kappa_P = 0.1
    vor2.a = a
    vor2.k = k

    vor2.simulate2(progress_bar=False, print_updates=False);

    vor2.animate(file_name=prefix, n_frames=100, print_updates=False)

100%|██████████| 2/2 [03:20<00:00, 100.09s/it]


<hr>